<img src=https://i.ytimg.com/vi/knxlmCVFAZI/maxresdefault.jpg>

In [1]:
#!pip3 install twilio

In [2]:
import os
from twilio.rest import Client
from twilio_config import *
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime


# Armado de la URL

In [3]:
query = 'Salta'
api_key = API_KEY_WAPI

url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
url_clima

'http://api.weatherapi.com/v1/forecast.json?key=f3cf65ff7ae64bccad9210657220807&q=Salta&days=1&aqi=no&alerts=no'

In [4]:
response = requests.get(url_clima).json()

In [5]:
response

{'location': {'name': 'Salta',
  'region': 'Salta',
  'country': 'Argentina',
  'lat': -24.78,
  'lon': -65.42,
  'tz_id': 'America/Argentina/Salta',
  'localtime_epoch': 1708989791,
  'localtime': '2024-02-26 20:23'},
 'current': {'last_updated_epoch': 1708989300,
  'last_updated': '2024-02-26 20:15',
  'temp_c': 21.0,
  'temp_f': 69.8,
  'is_day': 0,
  'condition': {'text': 'Partly cloudy',
   'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png',
   'code': 1003},
  'wind_mph': 4.3,
  'wind_kph': 6.8,
  'wind_degree': 250,
  'wind_dir': 'WSW',
  'pressure_mb': 1016.0,
  'pressure_in': 30.0,
  'precip_mm': 0.12,
  'precip_in': 0.0,
  'humidity': 94,
  'cloud': 75,
  'feelslike_c': 21.0,
  'feelslike_f': 69.8,
  'vis_km': 9.0,
  'vis_miles': 5.0,
  'uv': 1.0,
  'gust_mph': 8.7,
  'gust_kph': 14.0},
 'forecast': {'forecastday': [{'date': '2024-02-26',
    'date_epoch': 1708905600,
    'day': {'maxtemp_c': 22.5,
     'maxtemp_f': 72.4,
     'mintemp_c': 18.1,
     'mintemp_f': 64.5

In [6]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [7]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [8]:
len(response['forecast']['forecastday'][0]['hour'])

24

In [16]:
response['forecast']['forecastday'][0]['hour'][1]

{'time_epoch': 1708920000,
 'time': '2024-02-26 01:00',
 'temp_c': 18.2,
 'temp_f': 64.8,
 'is_day': 0,
 'condition': {'text': 'Mist',
  'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png',
  'code': 1030},
 'wind_mph': 1.3,
 'wind_kph': 2.2,
 'wind_degree': 258,
 'wind_dir': 'WSW',
 'pressure_mb': 1016.0,
 'pressure_in': 29.99,
 'precip_mm': 0.07,
 'precip_in': 0.0,
 'snow_cm': 0.0,
 'humidity': 95,
 'cloud': 53,
 'feelslike_c': 18.2,
 'feelslike_f': 64.8,
 'windchill_c': 18.2,
 'windchill_f': 64.8,
 'heatindex_c': 18.2,
 'heatindex_f': 64.8,
 'dewpoint_c': 17.5,
 'dewpoint_f': 63.5,
 'will_it_rain': 1,
 'chance_of_rain': 100,
 'will_it_snow': 0,
 'chance_of_snow': 0,
 'vis_km': 2.0,
 'vis_miles': 1.0,
 'gust_mph': 2.2,
 'gust_kph': 3.5,
 'uv': 1.0}

In [19]:
response['forecast']['forecastday'][0]['hour'][0]['time'].split()[0] #Fecha

'2024-02-26'

In [22]:
int(response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0]) #Hora

1

In [24]:
response['forecast']['forecastday'][0]['hour'][0]['condition']['text'] #Condicion

'Mist'

In [25]:
response['forecast']['forecastday'][0]['hour'][0]['temp_c'] #Temperatura

18.8

In [26]:
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain'] #Llovera o no, 0 para no, 1 para si

1

In [32]:
response['forecast']['forecastday'][0]['hour'][1]['chance_of_rain'] #Posibilidad de lluvia

100

# Dataframe

In [33]:
def get_forecast(response,i):

    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0] #Fecha
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0]) #Hora
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text'] #Condicion
    tempe = response['forecast']['forecastday'][0]['hour'][i]['temp_c'] #Temperatura
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain'] #Llovera o no, 0 para no, 1 para si
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain'] #Posibilidad de lluvia

    return fecha,hora,condicion,tempe,rain,prob_rain

In [34]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):
#tqdm es para visualizar una barra de carga en las iteraciones del for
    datos.append(get_forecast(response,i))

100%|██████████| 24/24 [00:00<00:00, 24047.61it/s]


In [36]:
datos[0]

('2024-02-26', 0, 'Mist', 18.8, 1, 100)

In [38]:
col = ['Fecha','Hora','Condicion','Temperatura','Lluvia','prob_lluvia']
df = pd.DataFrame(datos,columns=col)

In [39]:
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2024-02-26,0,Mist,18.8,1,100
1,2024-02-26,1,Mist,18.2,1,100
2,2024-02-26,2,Light rain shower,18.1,1,100
3,2024-02-26,3,Light rain,18.4,1,100
4,2024-02-26,4,Light rain shower,18.6,1,100
5,2024-02-26,5,Light drizzle,18.5,1,100
6,2024-02-26,6,Fog,18.2,1,100
7,2024-02-26,7,Fog,18.4,0,61
8,2024-02-26,8,Fog,18.7,1,87
9,2024-02-26,9,Fog,19.0,1,100


In [44]:
df_rain = df[(df['Lluvia']==1) & (df['Hora']>6) & (df['Hora']< 23)]
df_rain = df_rain[['Hora','Condicion']]
df_rain.set_index('Hora', inplace = True)

In [45]:
df_rain

,Condicion
Hora,
8,Fog
9,Fog
10,Light rain shower
11,Light drizzle
12,Light rain shower
13,Mist
14,Mist
15,Patchy rain nearby
17,Patchy rain nearby


In [46]:
'\nHola! \n\n\n El pronostico del tiempo hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain)

'\nHola! \n\n\n El pronostico del tiempo hoy 2024-02-26 en Salta es : \n\n\n                Condicion\nHora                    \n8                    Fog\n9                    Fog\n10     Light rain shower\n11         Light drizzle\n12     Light rain shower\n13                  Mist\n14                  Mist\n15    Patchy rain nearby\n17    Patchy rain nearby\n18    Patchy rain nearby\n19         Light drizzle\n20         Partly cloudy\n21    Patchy rain nearby\n22         Light drizzle'

In [47]:
PHONE_NUMBER

'+1 614 401 3161'

# Mensaje SMS desde Twilio

In [48]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID 
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                     body='\nHola! \n\n\n El pronostico de lluvia hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain),
                     from_=PHONE_NUMBER,
                     to='+543875990930'
                 )

print('Mensaje Enviado ' + message.sid)

TwilioRestException: HTTP 401 error: Unable to create record: Authentication Error - invalid username

# Challenge 

* Extrae el valor del dolar en tu país y el top 10 de criptomonedas con su respectiva valoración
* Ahora envia un mensaje diarío a tu Whatsapp usando Twilio

**hint 💡** Investiga que API's gratuitas existen para consultar estos datos



<img src="WhatsApp Image 2022-09-13 at 9.12.18 AM.jpeg" width="200" height="200" />